In [259]:
!pip3 install xgboost

ERROR: Could not install packages due to an EnvironmentError: [WinError 32] 程序無法存取檔案，因為檔案正由另一個程序使用。: 'C:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpu\\Lib\\site-packages\\xgboost\\lib\\xgboost.dll'
Consider using the `--user` option or check the permissions.



In [406]:
import os
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# import seaborn as sns
# sns.set(style="ticks", color_codes=True)
# sns.set(rc={'figure.figsize':(10, 10)})
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures
le = LabelEncoder()
sc = MinMaxScaler()
sd = StandardScaler()
poly = PolynomialFeatures(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef

import tensorflow as tf
tf.__version__
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Embedding

from custom_functions import (
    gen_MonthDayHrMinDayOfWeek,
    gen_timeblock
)


'2.1.0'

In [676]:
user_attr = pd.read_csv('users.csv')
user_attr['attr_1'] = user_attr['attr_1'].fillna(-1)
user_attr['attr_2'] = user_attr['attr_2'].fillna(-1)
user_attr['age'] = user_attr['age'].fillna(-1)
user_attr['age'] = np.where(user_attr['age']<=15, 0, user_attr['age'])
user_attr['age'] = np.where((user_attr['age']>=65)&(user_attr['age']<118), 65, user_attr['age'])
# user_attr['domain'] = le.fit_transform(user_attr['domain'])
# user_attr['domain'] = user_attr['domain'].astype('int64')
user_attr['attr_1'] = user_attr['attr_1'].astype('int64').astype(str)
user_attr['attr_2'] = user_attr['attr_2'].astype('int64').astype(str)
user_attr['attr_3'] = user_attr['attr_3'].astype('int64').astype(str)
dummy_userAttr = pd.get_dummies(user_attr[['attr_1','attr_2','attr_3','domain']])
user_attr = pd.concat([user_attr, dummy_userAttr], axis=1)
user_attr.drop(['attr_1','attr_1_-1','attr_2','attr_2_-1','attr_3','attr_3_0','domain','domain_other'], axis=1, inplace=True)
user_attr['age'] = user_attr['age'].astype('int64')
# user_attr.info()
# user_attr.isnull().sum()
user_attr.shape
user_attr.head()

train = pd.read_csv('train.csv')
train['last_open_day'] = np.where(train['last_open_day']=='Never open', 808*10, train['last_open_day'])
train['last_login_day'] = np.where(train['last_login_day']=='Never login', 18141*10, train['last_login_day'])
train['last_checkout_day'] = np.where(train['last_checkout_day']=='Never checkout', 1445*10, train['last_checkout_day'])
train['last_open_day'] = train['last_open_day'].astype('int64')
train['last_login_day'] = train['last_login_day'].astype('int64')
train['last_checkout_day'] = train['last_checkout_day'].astype('int64')
train = gen_MonthDayHrMinDayOfWeek(train, 'grass_date')
train.drop(['month','day','hour','minute'], axis=1, inplace=True)
train['country_code'] = train['country_code'].astype(str)
dummy_train = pd.get_dummies(train[['country_code']])
train = pd.concat([train, dummy_train], axis=1)
train.drop(['country_code','country_code_7'], axis=1, inplace=True)
# train.info()
# train.isnull().sum()
train.shape
train.head()
Y = train[['row_id','open_flag']]


train_userAttr = pd.merge(train, user_attr, on='user_id', how='left')
train_userAttr.shape
train_userAttr.head()
train_userAttr.groupby(['open_flag']).size()
# train_userAttr.isnull().sum()


test = pd.read_csv('test.csv')
# test = test.sort_values('user_id')
test['last_open_day'] = np.where(test['last_open_day']=='Never open', 99*10, test['last_open_day'])
test['last_login_day'] = np.where(test['last_login_day']=='Never login', 996*10, test['last_login_day'])
test['last_checkout_day'] = np.where(test['last_checkout_day']=='Never checkout', 997*10, test['last_checkout_day'])
test['last_open_day'] = test['last_open_day'].astype('int64')
test['last_login_day'] = test['last_login_day'].astype('int64')
test['last_checkout_day'] = test['last_checkout_day'].astype('int64')
test = gen_MonthDayHrMinDayOfWeek(test, 'grass_date')
test.drop(['month','day','hour','minute'], axis=1, inplace=True)
test['country_code'] = test['country_code'].astype(str)
dummy_test = pd.get_dummies(test[['country_code']])
test = pd.concat([test, dummy_test], axis=1)
test.drop(['country_code','country_code_7'], axis=1, inplace=True)
test.info()
# test.isnull().sum()
test.shape
test.head()

test_userAttr = pd.merge(test, user_attr, on='user_id', how='left')
test_userAttr.shape
test_userAttr.head()
# test_userAttr.isnull().sum()


submitSample = pd.read_csv('sample_submission_0_1.csv')
submitSample['row_id'] = test['row_id']
submitSample.shape
submitSample.head()



(127886, 20)

,user_id,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com
0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,50,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
2,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,4,33,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0


(73539, 23)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6
0,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0,2,0,0,0,1,0,0
1,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1,2,0,0,0,1,0,0
2,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2,2,0,0,0,0,0,1
3,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3,2,1,0,0,0,0,0
4,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4,2,0,0,0,0,0,1


(73539, 42)

...

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6
0,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0,2,0,0,0,0,0,1
1,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1,2,0,0,0,0,0,1
2,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2,2,0,0,0,0,1,0
3,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3,2,1,0,0,0,0,0
4,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4,2,0,0,0,0,1,0


(55970, 41)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com
0,0,35,27,2,13,2,3,4,10,34,134,0,6,18,0,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,130,35,7,5,383,1,1,1,5,5,5,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,150,25,34,1,3,0,0,0,13,19,38,2,2,2,2,2,0,0,0,0,1,0,33,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,181,36,63,5,5,0,0,0,43,110,173,2,5,5,3,2,1,0,0,0,0,0,22,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,192,23,6,5,54,0,0,0,4,12,39,0,0,2,4,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


(55970, 2)

,row_id,open_flag
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


#### domain_knowledge features
- `open_count_per10day_last_60_days`
- `open_count_per10day_last_30_days`
- `open_count_trend_60-30`
- `open_count_trend_30-10`

In [677]:
def open_count_features(train_userAttr):
    train_userAttr['open_count_per10day_last_60_days'] = train_userAttr['open_count_last_60_days'] / 6
    train_userAttr['open_count_per10day_last_30_days'] = train_userAttr['open_count_last_30_days'] / 3
    train_userAttr['open_count_trend_60-30'] = train_userAttr['open_count_per10day_last_30_days'] - train_userAttr['open_count_per10day_last_60_days']
    train_userAttr['open_count_trend_30-10'] = train_userAttr['open_count_last_10_days'] - train_userAttr['open_count_per10day_last_30_days']
    return train_userAttr

def login_count_features(train_userAttr):
    train_userAttr['login_count_per10day_last_60_days'] = train_userAttr['login_count_last_60_days'] / 6
    train_userAttr['login_count_per10day_last_30_days'] = train_userAttr['login_count_last_30_days'] / 3
    train_userAttr['login_count_trend_60-30'] = train_userAttr['login_count_per10day_last_30_days'] - train_userAttr['login_count_per10day_last_60_days']
    train_userAttr['login_count_trend_30-10'] = train_userAttr['login_count_last_10_days'] - train_userAttr['login_count_per10day_last_30_days']
    return train_userAttr

def checkout_count_features(train_userAttr):
    train_userAttr['checkout_count_per10day_last_60_days'] = train_userAttr['checkout_count_last_60_days'] / 6
    train_userAttr['checkout_count_per10day_last_30_days'] = train_userAttr['checkout_count_last_30_days'] / 3
    train_userAttr['checkout_count_trend_60-30'] = train_userAttr['checkout_count_per10day_last_30_days'] - train_userAttr['checkout_count_per10day_last_60_days']
    train_userAttr['checkout_count_trend_30-10'] = train_userAttr['checkout_count_last_10_days'] - train_userAttr['checkout_count_per10day_last_30_days']
    return train_userAttr

train_userAttr = open_count_features(train_userAttr)
test_userAttr = open_count_features(test_userAttr)
train_userAttr = login_count_features(train_userAttr)
test_userAttr = login_count_features(test_userAttr)
train_userAttr = checkout_count_features(train_userAttr)
test_userAttr = checkout_count_features(test_userAttr)


#### Polynomial Features 
- https://heartbeat.fritz.ai/a-practical-guide-to-feature-engineering-in-python-8326e40747c8


In [678]:
def polynomial_feature(train_userAttr, numeric_only=True):
    if numeric_only == True:
        selected_col = ['open_count_last_10_days','open_count_last_30_days','open_count_last_60_days']
    else:
        print('crossing all features...')
        selected_col = [c for c in train_userAttr.columns if c not in ['open_flag','user_id','row_id']]
        
    v = poly.fit_transform(train_userAttr[selected_col])
    left_col = [c for c in train_userAttr if c not in selected_col]
    return v, left_col


poly_features, left_col = polynomial_feature(train_userAttr, numeric_only=True)
poly_features = pd.DataFrame(poly_features, columns=['poly_features_' + str(x) for x in range(0, poly_features.shape[1])])

train_userAttr = pd.concat([train_userAttr[left_col], poly_features], axis=1)
train_userAttr.shape
train_userAttr.head()


poly_features, left_col = polynomial_feature(test_userAttr, numeric_only=True)
poly_features = pd.DataFrame(poly_features, columns=['poly_features_' + str(x) for x in range(0, poly_features.shape[1])])

test_userAttr = pd.concat([test_userAttr[left_col], poly_features], axis=1)
test_userAttr.shape
test_userAttr.head()


(73539, 61)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
0,43,44,19,6,18,12,43,99,0,5,10,0,0,2,0,0,0,1,0,0,47,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.666667,0.666667,0.000000,-0.666667,16.500000,14.333333,-2.166667,-2.333333,1.666667,1.666667,0.000000,-1.666667,1.0,0.0,2.0,4.0,0.0,0.0,0.0,4.0,8.0,16.0
1,102,44,9,4,8,18,48,90,1,1,4,1,1,2,0,0,0,1,0,0,25,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,2.833333,3.000000,0.166667,-1.000000,15.000000,16.000000,1.000000,2.000000,0.666667,0.333333,-0.333333,0.666667,1.0,2.0,9.0,17.0,4.0,18.0,34.0,81.0,153.0,289.0
2,177,49,14,5,5,24,69,119,5,19,27,0,2,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.000000,1.333333,-0.666667,-1.333333,19.833333,23.000000,3.166667,1.000000,4.500000,6.333333,1.833333,-1.333333,1.0,0.0,4.0,12.0,0.0,0.0,0.0,16.0,48.0,144.0
3,184,49,49,9,53,9,23,69,1,3,6,0,3,2,1,0,0,0,0,0,24,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.166667,0.000000,-0.166667,0.000000,11.500000,7.666667,-3.833333,1.333333,1.000000,1.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,221,49,227,6,221,2,5,5,0,0,0,0,4,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.833333,1.666667,0.833333,0.333333,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(55970, 60)

,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,row_id,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
0,0,35,27,2,13,10,34,134,0,6,18,0,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.666667,1.000000,0.333333,1.000000,22.333333,11.333333,-11.000000,-1.333333,3.000000,2.000000,-1.000000,-2.000000,1.0,2.0,3.0,4.0,4.0,6.0,8.0,9.0,12.0,16.0
1,130,35,7,5,383,5,5,5,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.166667,0.333333,0.166667,0.666667,0.833333,1.666667,0.833333,3.333333,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,150,25,34,1,3,13,19,38,2,2,2,2,2,0,0,0,0,1,0,33,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,6.333333,6.333333,0.000000,6.666667,0.333333,0.666667,0.333333,1.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181,36,63,5,5,43,110,173,2,5,5,3,2,1,0,0,0,0,0,22,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.000000,0.000000,0.000000,0.000000,28.833333,36.666667,7.833333,6.333333,0.833333,1.666667,0.833333,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,192,23,6,5,54,4,12,39,0,0,2,4,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,6.500000,4.000000,-2.500000,0.000000,0.333333,0.000000,-0.333333,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### make train/test data


In [679]:
# train_userAttr.head()
train_target = train_userAttr.iloc[:, np.r_[1:11, 13:61]]
# train_target = train_userAttr.iloc[:, np.r_[1, 5:14, 15:54]]
# train_target = train_userAttr.iloc[:, 3:train_userAttr.shape[1]]
train_target.head()
Y.head()


,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
0,44,19,6,18,12,43,99,0,5,10,2,0,0,0,1,0,0,47,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.666667,0.666667,0.000000,-0.666667,16.500000,14.333333,-2.166667,-2.333333,1.666667,1.666667,0.000000,-1.666667,1.0,0.0,2.0,4.0,0.0,0.0,0.0,4.0,8.0,16.0
1,44,9,4,8,18,48,90,1,1,4,2,0,0,0,1,0,0,25,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,2.833333,3.000000,0.166667,-1.000000,15.000000,16.000000,1.000000,2.000000,0.666667,0.333333,-0.333333,0.666667,1.0,2.0,9.0,17.0,4.0,18.0,34.0,81.0,153.0,289.0
2,49,14,5,5,24,69,119,5,19,27,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.000000,1.333333,-0.666667,-1.333333,19.833333,23.000000,3.166667,1.000000,4.500000,6.333333,1.833333,-1.333333,1.0,0.0,4.0,12.0,0.0,0.0,0.0,16.0,48.0,144.0
3,49,49,9,53,9,23,69,1,3,6,2,1,0,0,0,0,0,24,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.166667,0.000000,-0.166667,0.000000,11.500000,7.666667,-3.833333,1.333333,1.000000,1.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,49,227,6,221,2,5,5,0,0,0,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.833333,1.666667,0.833333,0.333333,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,row_id,open_flag
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [465]:
# train_target = pd.read_csv('train_target_with_PCA_features.csv')
# train_target.head()
# test_target = pd.read_csv('test_target_with_PCA_features.csv')
# test_target.head()


#### data split

In [680]:
X = train_target
print(X.shape)
print(Y.shape)

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.35, random_state=832, stratify=Y.open_flag)
print('train sets: ', train_X.shape[0], '\ntest sets: ', test_X.shape[0])

train_x, val_x, train_y, val_y = train_test_split(train_X, train_Y, test_size=0.25, random_state=383, stratify=train_Y.open_flag)
print('train sets: ', train_x.shape[0], '\nval sets: ', val_x.shape[0])


(73539, 58)
(73539, 2)
train sets:  47800 
test sets:  25739
train sets:  35850 
val sets:  11950


In [681]:
## if not sampling
# train_x.drop(['row_id'], axis=1, inplace=True)
train_x.head()
train_x = train_x.values
train_x.shape
train_y = train_y.open_flag
train_y.shape

# val_x.drop(['row_id'], axis=1, inplace=True)
val_x.head()
val_x = val_x.values
val_x.shape
val_y = val_y.open_flag
val_y.shape


,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
4067,61,2,1,10,9,38,73,1,5,5,4,1,0,0,0,0,0,24,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,5.833333,6.333333,0.500000,2.666667,12.166667,12.666667,0.500000,-3.666667,0.833333,1.666667,0.833333,-0.666667,1.0,9.0,19.0,35.0,81.0,171.0,315.0,361.0,665.0,1225.0
54231,36,11,15,56,0,12,32,0,0,1,3,0,1,0,0,0,0,20,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,3.000000,2.333333,-0.666667,-2.333333,5.333333,4.000000,-1.333333,-4.000000,0.166667,0.000000,-0.166667,0.000000,1.0,0.0,7.0,18.0,0.0,0.0,0.0,49.0,126.0,324.0
22722,36,8080,13,135,6,9,9,0,0,0,2,0,1,0,0,0,0,20,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1.500000,3.000000,1.500000,3.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41919,24,3,2,3,13,69,161,1,7,15,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3.000000,3.000000,0.000000,-2.000000,26.833333,23.000000,-3.833333,-10.000000,2.500000,2.333333,-0.166667,-1.333333,1.0,1.0,9.0,18.0,1.0,9.0,18.0,81.0,162.0,324.0
73449,38,14,2,14450,7,20,20,0,1,1,1,0,1,0,0,0,0,57,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,3.333333,6.666667,3.333333,0.333333,0.166667,0.333333,0.166667,-0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(35850, 58)

(35850,)

,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
6082,32,8080,24,14450,0,1,1,0,0,0,5,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.166667,0.333333,0.166667,-0.333333,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24685,40,21,1,44,15,52,136,0,0,14,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.333333,0.666667,0.333333,-0.666667,22.666667,17.333333,-5.333333,-2.333333,2.333333,0.000000,-2.333333,0.000000,1.0,0.0,2.0,2.0,0.0,0.0,0.0,4.0,4.0,4.0
37880,60,71,1,1,19,46,95,2,2,3,5,1,0,0,0,0,0,23,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,15.833333,15.333333,-0.500000,3.666667,0.500000,0.666667,0.166667,1.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3011,39,2,5,39,6,20,47,0,0,1,3,0,1,0,0,0,0,34,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,3.000000,3.333333,0.333333,3.666667,7.833333,6.666667,-1.166667,-0.666667,0.166667,0.000000,-0.166667,0.000000,1.0,7.0,10.0,18.0,49.0,70.0,126.0,100.0,180.0,324.0
70784,64,23,8,8,2,6,9,1,1,1,1,0,0,1,0,0,0,41,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.000000,1.333333,0.333333,-1.333333,1.500000,2.000000,0.500000,0.000000,0.166667,0.333333,0.166667,0.666667,1.0,0.0,4.0,6.0,0.0,0.0,0.0,16.0,24.0,36.0


(11950, 58)

(11950,)

In [682]:
test_target = test_userAttr.iloc[:, np.r_[1:11, 12:60]]
# test_target = test_userAttr.iloc[:, 2:test_userAttr.shape[1]]
test_target.head()
test_target.shape


,subject_line_length,last_open_day,last_login_day,last_checkout_day,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,day_of_week,country_code_1,country_code_2,country_code_3,country_code_4,country_code_5,country_code_6,age,attr_1_0,attr_1_1,attr_2_0,attr_2_1,attr_3_1,attr_3_2,attr_3_3,attr_3_4,domain_@163.com,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,open_count_per10day_last_60_days,open_count_per10day_last_30_days,open_count_trend_60-30,open_count_trend_30-10,login_count_per10day_last_60_days,login_count_per10day_last_30_days,login_count_trend_60-30,login_count_trend_30-10,checkout_count_per10day_last_60_days,checkout_count_per10day_last_30_days,checkout_count_trend_60-30,checkout_count_trend_30-10,poly_features_0,poly_features_1,poly_features_2,poly_features_3,poly_features_4,poly_features_5,poly_features_6,poly_features_7,poly_features_8,poly_features_9
0,35,27,2,13,10,34,134,0,6,18,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.666667,1.000000,0.333333,1.000000,22.333333,11.333333,-11.000000,-1.333333,3.000000,2.000000,-1.000000,-2.000000,1.0,2.0,3.0,4.0,4.0,6.0,8.0,9.0,12.0,16.0
1,35,7,5,383,5,5,5,0,0,0,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.166667,0.333333,0.166667,0.666667,0.833333,1.666667,0.833333,3.333333,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,25,34,1,3,13,19,38,2,2,2,2,0,0,0,0,1,0,33,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,6.333333,6.333333,0.000000,6.666667,0.333333,0.666667,0.333333,1.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36,63,5,5,43,110,173,2,5,5,2,1,0,0,0,0,0,22,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.000000,0.000000,0.000000,0.000000,28.833333,36.666667,7.833333,6.333333,0.833333,1.666667,0.833333,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23,6,5,54,4,12,39,0,0,2,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,6.500000,4.000000,-2.500000,0.000000,0.333333,0.000000,-0.333333,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(55970, 58)

#### down-sampling

In [133]:
def down_sampling(times, train_x, train_y, label_col='open_flag', id_col='row_id'):

    train_1_rowID = train_y[train_y[label_col]==1][id_col].values
    print('num of pos label: ', len(train_1_rowID))
    print('origin num of neg label: ', len(train_y[train_y[label_col]==0]))
    train_0_sampled_rowID = train_y[train_y[label_col]==0].sample(int(len(train_1_rowID)*times))[id_col].values

    train_sampled_rowID = []
    for i in train_0_sampled_rowID:
        train_sampled_rowID.append(i)
    for i in train_1_rowID:
        train_sampled_rowID.append(i)
    print(len(train_sampled_rowID))
    train_x = train_x[train_x[id_col].isin(train_sampled_rowID)]
    train_x.drop([id_col], axis=1, inplace=True)
    train_x = train_x.values
    print(train_x.shape)
    train_y = train_y[train_y[id_col].isin(train_sampled_rowID)][label_col].values
    print(train_y.shape)
    
    return train_x, train_y


train_x, train_y = down_sampling(2.5, train_x, train_y)
val_x, val_y = down_sampling(2.5, val_x, val_y)


num of pos label:  5584
origin num of neg label:  30266
19544
(19544, 51)
(19544,)
num of pos label:  1862
origin num of neg label:  10088
6517
(6517, 51)
(6517,)


C:\ProgramData\Anaconda3\envs\tensorflow_gpu\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### scaling after split


In [683]:
# for c in train_userAttr.columns:
#     if c in ['subject_line_length', 'last_open_day', 'last_login_day', 'last_checkout_day', 'open_count_last_10_days', 
#              'open_count_last_30_days', 'open_count_last_60_days', 'login_count_last_10_days', 
#              'login_count_last_30_days', 'login_count_last_60_days', 'checkout_count_last_10_days', 
#              'checkout_count_last_30_days', 'checkout_count_last_60_days', 'age', 'day_of_week', 'open_count_per10day_last_60_days',
#              'login_count_per10day_last_60_days', 'checkout_count_per10day_last_60_days', 'open_count_per10day_last_30_days',
#              'login_count_per10day_last_30_days', 'checkout_count_per10day_last_30_days', 'open_count_trend_60-30',
#              'login_count_trend_60-30', 'checkout_count_trend_60-30', 'open_count_trend_30-10',
#              'login_count_trend_30-10', 'checkout_count_trend_30-10']:
#         train_x[c] = sd.fit(train_x[c].values.reshape(-1,1))
#         train_x[c] = sd.transform(train_x[c].values.reshape(-1,1))
#         val_x[c] = sd.transform(val_x[c].values.reshape(-1,1))

sd.fit(train_x)
train_x = sd.transform(train_x)
train_x.shape
val_x = sd.transform(val_x)
val_x.shape
test_target = sd.transform(test_target)
test_target.shape


StandardScaler()

(35850, 58)

(11950, 58)

(55970, 58)

#### model tuning

In [900]:
produce_date = 200806
model_name = 'NN'
batch_size = 1024
epochs = 15
class_weight = {0: 0.9, 1: 3}
model_desc = f'batch_size_{batch_size}-epochs_{epochs}-class_weight_{str(class_weight)}'
feature_desc = 'standardlizedScaled_&_polynomial_features_on_last_xxx_day'


In [901]:
# model = load_model()

In [965]:
model = Sequential()
model.add(Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00015), input_shape=(train_x.shape[1],)))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00015)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# fit the model
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, class_weight=class_weight, verbose=1)


  ...
    to  
  ['...']
Train on 35850 samples
Epoch 1/15
35850/35850 [==============================] - 0s 13us/sample - loss: 0.7488 - accuracy: 0.7701
Epoch 2/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.6690 - accuracy: 0.8389
Epoch 3/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.6318 - accuracy: 0.8406
Epoch 4/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.6218 - accuracy: 0.8416
Epoch 5/15
35850/35850 [==============================] - 0s 2us/sample - loss: 0.6087 - accuracy: 0.8402
Epoch 6/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.6047 - accuracy: 0.8442
Epoch 7/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.5984 - accuracy: 0.8436
Epoch 8/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.5935 - accuracy: 0.8446
Epoch 9/15
35850/35850 [==============================] - 0s 3us/sample - loss: 0.5933 - accuracy: 0.84

In [966]:
# evaluate the model
loss, acc = model.evaluate(val_x, val_y, verbose=0)
print('Val Accuracy: %.3f' % acc)

y_pred = model.predict(val_x)
y_pred = [1 if x >= 0.5 else 0 for x  in y_pred]
print(classification_report(y_true=val_y, y_pred=y_pred))
print(confusion_matrix(y_true=val_y, y_pred=y_pred))
print('AUC score: ', roc_auc_score(y_true=val_y, y_score=y_pred))
print('MCC score: ', matthews_corrcoef(y_true=val_y, y_pred=y_pred))


Val Accuracy: 0.845
              precision    recall  f1-score   support

           0       0.94      0.87      0.91     10088
           1       0.50      0.68      0.58      1862

    accuracy                           0.85     11950
   macro avg       0.72      0.78      0.74     11950
weighted avg       0.87      0.85      0.85     11950

[[8826 1262]
 [ 590 1272]]
AUC score:  0.7790186423916368
MCC score:  0.4951297956283381


In [967]:
y_pred = model.predict(test_X.values)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print(classification_report(y_true=test_Y.open_flag, y_pred=y_pred))
print(confusion_matrix(y_true=test_Y.open_flag, y_pred=y_pred))
print('AUC score: ', roc_auc_score(y_true=test_Y.open_flag, y_score=y_pred))
print('MCC score: ', matthews_corrcoef(y_true=test_Y.open_flag, y_pred=y_pred))
model_desc = f'{model_desc}--local_MCC_{round(matthews_corrcoef(y_true=test_Y.open_flag, y_pred=y_pred), 4)}'


              precision    recall  f1-score   support

           0       0.93      0.35      0.51     21729
           1       0.20      0.86      0.32      4010

    accuracy                           0.43     25739
   macro avg       0.56      0.60      0.41     25739
weighted avg       0.82      0.43      0.48     25739

[[ 7595 14134]
 [  571  3439]]
AUC score:  0.6035694336802846
MCC score:  0.1614083549040279


#### prediction

In [288]:
test_x = test_target.values
test_x.shape
target_pred = model.predict(test_x)
target_pred = [1 if x >= 0.5 else 0 for x in target_pred]
submitSample['open_flag'] = target_pred
submitSample.head()
submitSample.groupby(['open_flag']).size()


(55970, 26)

,row_id,open_flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


open_flag
0    50717
1     5253
dtype: int64

In [289]:
submitSample.to_csv(f'submission_{produce_date}__{model_name}_{model_desc}_feature-{feature_desc}.csv', index=False)
